In [1]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

import io
post_covid_df = pd.read_csv(io.BytesIO(uploaded['Project_Data_Post-Covid.csv']))

Saving Project_Data_Post-Covid.csv to Project_Data_Post-Covid.csv


In [2]:
post_covid_df.head()

,product_category,product_url,review_date,Date,handle,rating,helpfulness_rating,review
0,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,4/2/2021,Post_Covid,Den,1,NaN,...even with short trimmed nails. I use dispos...
1,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,11/17/2020,Post_Covid,patricia43,5,NaN,...they offer the protection I need for chores...
2,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,9/4/2020,Post_Covid,SU,1,NaN,1 out of every 2 gloves rips just from the for...
3,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,8/13/2020,Post_Covid,Sheila M.,1,NaN,1. They are very small. 2. They are so thin th...
4,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,8/26/2020,Post_Covid,casebes,1,NaN,1st glove ripped as i put it on. Ordered mediu...


In [3]:
post_covid_df = post_covid_df.drop(columns=['product_url', 'helpfulness_rating'], axis=1)

In [4]:
post_covid_df['rating'] = post_covid_df['rating'].apply(lambda z:'positive' if z>3 else 'negative')

In [5]:
post_covid_df.head()

,product_category,review_date,Date,handle,rating,review
0,disposable_gloves,4/2/2021,Post_Covid,Den,negative,...even with short trimmed nails. I use dispos...
1,disposable_gloves,11/17/2020,Post_Covid,patricia43,positive,...they offer the protection I need for chores...
2,disposable_gloves,9/4/2020,Post_Covid,SU,negative,1 out of every 2 gloves rips just from the for...
3,disposable_gloves,8/13/2020,Post_Covid,Sheila M.,negative,1. They are very small. 2. They are so thin th...
4,disposable_gloves,8/26/2020,Post_Covid,casebes,negative,1st glove ripped as i put it on. Ordered mediu...


In [6]:
import nltk

nltk.download('punkt')

import string
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
post_covid_df['review'] = post_covid_df['review'].astype(str)

In [8]:
from nltk.tokenize import word_tokenize

for item in post_covid_df['review']:
  words = word_tokenize(item.replace('\n',' '))

In [9]:
clean = [word for word in words if word not in set(string.punctuation)]

In [10]:
characters_to_remove = ["''",'``','!',"'"]
clean_words = [word for word in clean if word not in set(characters_to_remove)]

In [11]:
nltk.download('stopwords')

from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
clean = [word for word in clean_words if word not in english_stops]

In [13]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [14]:
reviews = [(list(clean), rating)
              for review_text in post_covid_df['review']
              for rating in post_covid_df['rating']]

random.shuffle(reviews)

In [15]:
clean = [wordnet_lemmatizer.lemmatize(word) for word in clean]

In [16]:
all_words = nltk.FreqDist(word.lower() for word in clean)
word_features = list(all_words)

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [17]:
split = 0.7

featuresets = [(document_features(d), c) for (d,c) in reviews]

train_set, test_set = featuresets[int(round(len(featuresets)*split,0))-1:], featuresets[:int(round(len(featuresets)*split,0))-1]

nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [18]:
print(nltk.classify.accuracy(nb_classifier, test_set))

0.5793322188971333


In [23]:
nb_classifier.show_most_informative_features(n=5)

Most Informative Features
      contains(anything) = True           negati : positi =      1.0 : 1.0
         contains(glove) = False          negati : positi =      1.0 : 1.0
           contains(...) = True           negati : positi =      1.0 : 1.0
          contains(good) = True           negati : positi =      1.0 : 1.0
           contains(use) = True           negati : positi =      1.0 : 1.0
